<img src="inundation_banner.png" width="100%" />
<font face="Arial">
<br><font size="7"> SAR Application: Inundation Temporal Dyanimcs </font>
<img src="NISAR_Mission_Logo.png" width="250" align="right" />
<font size="6"> L-Band SAR Flooding and Water State Change<font color='rgba(200,0,0,0.2)'> </font>
<br><font size="5"> I. How to create SAR quicklook images of flooding? <font color='rgba(200,0,0,0.2)'> </font>    
<br><font size="3"> PolSAR HH and HV (and VV if possible) are useful products for flood area extent. For viewing, Pauli decomposition can be used for quad-pol, or HH/HV and VV/VH for dual-pol images. </font>
<br><font size="5"> II. How to interpret SAR images of flooding?<font color='rgba(200,0,0,0.2)'> </font>
<br><font size="3"> To interpret SAR images of flooding, compare the RGB image before and after the flood. L-band brightness increases over flooded forests and tall vegetation, while short vegetation and open water are dark. Co-pol signal is relatively stronger in inundated areas owing to double-bounce, and Co-pol signal component increases in areas of flood expansion. </font>

<font size="5"> Hurricane Florence UAVSAR Example<font color='rgba(200,0,0,0.2)'> </font> 
<br><font size="3"> Hurricane Florence was a category 4 hurricane (August 31 - September 18, 2018) that caused significant freshwater and storm surge flooding along the southeastern coast of the Carolinas. From September 17 through September 23, UAVSAR (L-Band airborne radar) flew five times over the Pee Dee River in South Carolina to collect data during and after the hurricane. This interactive map displays two Pauli RGB UAVSAR images of line peedee_15100, acquired September 17 and September 23. Between these six days, noticeable differences occurred in inundation and open water extent along the river and in nearby fields. 
    
The overlay of the intensities of the different polarization channels, allows users to visually classify a scene by its backscattering mechanism, such as surface scattering (strong HH and VV return), volume scattering (strong HV return) and double-bounce scattering (strong HH return). Thus in RGB images, areas dominated by green (HV) intensity are typically vegetated areas. Areas dominated by shades of pink (HH+HV) intensity are typically inundated forests or vegetated fields. Black and dark grey areas are usually smooth surfaces (roads, open water, smooth bare ground) where there is very little radar backscatter.<font color='rgba(200,0,0,0.2)'> </font>

In [2]:
from ipyleaflet import (Map, basemaps, basemap_to_tiles, SplitMapControl, ImageOverlay, ScaleControl,
                        projections, LayersControl, FullScreenControl, LocalTileLayer, LegendControl)
import ipywidgets as widgets
from traitlets import Bool
from ipywidgets import AppLayout, Button, Layout, HTML
from pathlib import Path
from typing import *
import os

In [3]:
# Interactive Map Inputs
image1 = "peedee17/{z}/{x}/{y}.png"
image2 = "peedee23/{z}/{x}/{y}.png"

class CustomLocalLayer(LocalTileLayer):
    tms = Bool(True).tag(sync=True, o=True)

m = Map(basemap=basemap_to_tiles(basemaps.Esri.WorldImagery),
    center=(33.85, -79.35),
    zoom=9
    
)

layer = CustomLocalLayer(tms=True, path=image1, name="August 17 (During Hurricane)")
layer2 = CustomLocalLayer(tms=True, path=image2, name="August 23 (Post Hurricane)")

## Ipyleaflet Controls
control = SplitMapControl(left_layer=layer, right_layer=layer2)
legend = LegendControl({"HH-VV (Double Bounce)":"#FF0000", "HV + VH (Volume Scattering)":"#008000", "HH+VV (Specular Scattering)":"#0000FF"}, name="RGB Channels", position="topright")
description = LegendControl({"Flooded or bare ground":"#261947", "Flooded Vegetation (double bounce)":"#e8cef5", "Vegetation":"#7eab91"}, name="Interpretation", position="topright")
m.add_control(legend)
m.add_control(description)
m.add_control(control)
m.add_control(LayersControl())
m.add_control(FullScreenControl())
m.add_control(ScaleControl(position='bottomleft', metric=True, imperial=False, maxwidth=500))

m

Map(center=[33.85, -79.35], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

<font size="5"> UAVSAR Image Time Series<font color='rgba(200,0,0,0.2)'> </font>
<font face="Arial"><br><font size="3"> Changes in flooding are evident in the RGB images, even between adjacent UAVSAR data acquisitions. This slideshow shows a subset of the image located near the Pee Dee River in an area of changing flood extent along the river and neighboring open fields.<font color='rgba(200,0,0,0.2)'> </font>

In [4]:
# Source: https://gist.github.com/JarnoRFB/63c8cc9407a905588f84047fd4975ddc
# Source: https://blog.jupyter.org/introducing-templates-for-jupyter-widget-layouts-f72bcb35a662

A = TypeVar('A')

class Cycle(Generic[A]):
    def __init__(self, values: List[A]):
        self._values = values
        self._max_idx = len(values) - 1
        self._current_idx = 0
    
    def next(self) -> A:
        if self._current_idx == self._max_idx:
            self._current_idx = 0
        else:
            self._current_idx += 1
        return self._values[self._current_idx]
    
    def previous(self) -> A:
        if self._current_idx == 0:
            self._current_idx = self._max_idx
        else:
            self._current_idx -= 1
        return self._values[self._current_idx]
    
    def __repr__(self):
        return f"Cycle({self._values})"
    
def image_viewer(img_dir: os.PathLike = "./subset", format: str = "png"):
    """Create an image viewer widget to view the image of a certain format inside a directory.
    
    Args:
        img_dir: The directory to look for images in.
        format: The image format to consider.
    """

    prev_button = widgets.Button(
        description="Prev", icon="backward", layout=Layout(width="50%", height="30%")
    )
    
    next_button = widgets.Button(
        description="Next", icon="forward", layout=Layout(width="50%", height="30%")
    )
    
    import os
    from datetime import datetime
    
    abs_img_dir = Path(img_dir).expanduser().absolute()
    image_files = list(abs_img_dir.glob(f"*.{format}"))
    image_files = sorted(image_files)
    
    #files = []
    #for infile in sorted(image_files): 
    #    base = os.path.basename(infile)
    #    files.append(base)

    
    if not image_files:
        raise ValueError(f"No image for format {format} found in directory {abs_img_dir}.")
    images = Cycle(image_files)
    image = widgets.Image(
        value=open(image_files[0], "rb").read(),
        format=format,
        width="90%",
        height="100%",
    )
    
    #header = widgets.HTML(f"<h2>{files[0]}</h2>", layout=Layout(height="auto"))


    footer = widgets.HTML(f"<h4>{image_files[0]}</h4>", layout=Layout(height="auto"))

    
    def update_image(filename: str):
        with open(filename, "rb") as f:
            image.value = f.read()
    
    #def update_header(filename: str):
    #    header.value = f"<h1>{filename}</h1>"
    
    def update_footer(filename: str):
        footer.value = f"<h4>{filename}</h4>"

        
    def update_widgets(filename: str):
        update_image(filename)
        update_footer(filename)
        
        
    def handle_next(button):
        update_widgets(images.next())
        
        
    def handle_prev(button):
        update_widgets(images.previous())


    prev_button.on_click(handle_prev)
    next_button.on_click(handle_next) 
    
    app = AppLayout(
        #header=header,
        left_sidebar=prev_button,
        center=image,
        right_sidebar=next_button,
        footer=footer,
        justify_items="center",
        align_items="center",
    )
    
    return app

image_viewer()

ValueError: No image for format png found in directory /Users/peacock/Desktop/UAVSAR_Quicklooks/leaflet_test/subset.